In [ ]:
import numpy as np
import pandas as pd
import re


In [ ]:
# https://data.census.gov/cedsci/
df = pd.read_csv(
    "ACSDT5Y2020.B08006_2022-04-05T233519/ACSDT5Y2020.B08006_data_with_overlays_2022-03-18T113810.csv", header=1, index_col="id")

data = pd.DataFrame(index=df.index)

data["city"] = df["Geographic Area Name"]
data["city"] = data["city"].apply(lambda x: re.sub(r" [a-z]+(.+)?,", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r" [A-Z]+?,", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r"\/(.+?),", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r"\-(.+?),", ",", x))

data["bicycle"] = df["Estimate!!Total:!!Bicycle"].values
# drop all rows where column bicycle is zero
data.drop(data[data["bicycle"] == 0].index, inplace=True)

# data.loc["1600000US4752006", "city"] = "Nashville, Tennessee"
# data.loc["1600000US2148006", "city"] = "Louisville, Kentucky"
# data.loc["1600000US1571550", "city"] = "Honolulu, Hawaii"
# data.loc["1600000US2146027", "city"] = "Lexington, Kentucky"
# data.loc["1600000US1608830", "city"] = "Boise, Idaho"
# data.loc["1600000US1304204", "city"] = "Augusta, Georgia"

print(data.to_string())


In [ ]:
df = pd.read_csv(
    "ACSST5Y2020.S0101_2022-04-07T201744/ACSST5Y2020.S0101_data_with_overlays_2022-03-24T112908.csv", header=1, index_col="id")

data["population"] = df.loc[data.index,
                            "Estimate!!Total!!Total population"].values
data.drop(data[data["population"] < 50_000].index, inplace=True)


In [ ]:
data["per_100k"] = data["bicycle"] / data["population"] * 100_000
data.dropna(inplace=True)

chicago = data.loc["1600000US1714000", "per_100k"]

data["score"] = (data["per_100k"]/chicago).apply(np.log2)
data.sort_values(by="score", ascending=False, inplace=True)


In [ ]:
print(data.to_string())
data.to_csv("biking.csv")
